In [1]:
from utils import get_smiles_encodings, load_data, smile_to_hot
from model_regression import LinearRegression_RidgeRegression
from sklearn.linear_model import SGDRegressor
from datetime import datetime
import numpy as np
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
'''https://medium.com/coinmonks/support-vector-regression-or-svr-8eb3acf6d0ff'''

In [2]:
file_smiles = './dataset/QM9.txt'
file_properties = './dataset/properties_QM9.npz'
smiles, alphabet, largest_molecule_len = get_smiles_encodings(file_smiles)
properties = np.load(file_properties)['properties'].astype(np.float32)

X_train, X_test, y_train, y_test = load_data(smiles, properties)
print("X - train : ", X_train.shape)
print("y - train : ", y_train.shape)
X_hot = np.array([smile_to_hot(x, largest_molecule_len, alphabet)[1].sum(axis=0) for x in X_train])
#print("X_hot", X_hot)
X_hot_test = np.array([smile_to_hot(x, largest_molecule_len, alphabet)[1].sum(axis=0) for x in X_test])
#print("X_hot_test", X_hot_test)
print("X_hot : ", X_hot.shape)
print("X_hot_test : ", X_hot_test.shape)
print("y_train", y_train)
print("y_test",y_test)

X - train :  (92428,)
y - train :  (92428, 4)
X_hot :  (92428, 14)
X_hot_test :  (39612, 14)
y_train [[  0.6361    0.       16.043     0.     ]
 [  0.162     0.       17.031     0.     ]
 [ -0.8247    0.       18.015     0.     ]
 ...
 [ -0.5388    0.      130.143     1.     ]
 [ -1.7719    0.      131.131     1.     ]
 [ -0.89423   0.      131.131     1.     ]]
y_test [[ -1.3449   0.     132.115    1.    ]
 [  1.9317   0.     124.183    1.    ]
 [  1.0462   0.     125.171    1.    ]
 ...
 [ -0.8808   0.     120.155    8.    ]
 [  0.3187   0.     119.167    8.    ]
 [ -0.55     0.     121.139    8.    ]]


In [3]:
#n_samples, n_features = 10, 5
#rng = np.random.RandomState(0)
#y = rng.randn(n_samples)
#X = rng.randn(n_samples, n_features)
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_hot, y_train[:,0])
y_pred_0 = regr.predict(X_hot_test)
print("done : y_pred_0")
print("y_pred_0",y_pred_0)
print("y_pred_0 len", len(y_pred_0))


done : y_pred_0
y_pred_0 [-1.04298227  1.8759916   1.24370983 ... -0.63408629  0.36238152
 -0.53342194]
y_pred_0 len 39612


In [4]:
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_hot, y_train[:,1])
y_pred_1 = regr.predict(X_hot_test)

print("done : y_pred_1")

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_hot, y_train[:,2])
y_pred_2 = regr.predict(X_hot_test)

print("done : y_pred_2")

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_hot, y_train[:,3])
y_pred_3 = regr.predict(X_hot_test)
print("done : y_pred_3")



done : y_pred_1
done : y_pred_2
done : y_pred_3


In [ ]:
way too much time 
'''C_rate = [0.01,0.1,1.0,10.0,100.0]
kernel_function = ["linear", "poly", "rbf", "sigmoid", "precomputed"]
epsilon_rate = [0.01, 0.1, 1.0, 10.0, 100.0]
best_0 = [100,0,0,0]
best_1 = [100,0,0,0]
best_2 = [100,0,0,0]
best_3 = [100,0,0,0]

#we iterate over loss function and learning rates
for kernel in kernel_function:
    for C in C_rate:
        for eps in epsilon_rate:
            # 0 
            regr = make_pipeline(StandardScaler(), SVR(C=C, epsilon=eps, kernel=kernel))
            regr.fit(X_hot, y_train[:,0])
            y_pred_0 = regr.predict(X_hot_test)

            # 1
            regr = make_pipeline(StandardScaler(), SVR(C=C, epsilon=eps, kernel=kernel))
            regr.fit(X_hot, y_train[:,1])
            y_pred_1 = regr.predict(X_hot_test)
            y_pred_1 = np.round(y_pred_1)

            # 2 
            regr = make_pipeline(StandardScaler(), SVR(C=C, epsilon=eps, kernel=kernel))
            regr.fit(X_hot, y_train[:,2])
            y_pred_2 = regr.predict(X_hot_test)

            # 3 
            regr = make_pipeline(StandardScaler(), SVR(C=C, epsilon=eps, kernel=kernel))
            regr.fit(X_hot, y_train[:,3])
            y_pred_3 = regr.predict(X_hot_test)
            y_pred_3 = np.round(y_pred_3)

            # calcul res
            sum_0 = 0
            sum_1 = 0
            sum_2 = 0
            sum_3 = 0
            for i in range(y_test_0.shape[0]):

                # 0
                sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))

                # 1
                if(y_pred_1[i] == y_test_1[i]):
                    sum_1 = sum_1 + 1

                # 2
                sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))

                # 3
                if(y_pred_3[i] == y_test_3[i]):
                    sum_3 = sum_3 + 1

            sum_0 = sum_0 / y_test_0.shape[0]
            sum_1_r = 100 - (sum_1 / y_test_1.shape[0] * 100)
            sum_2 = sum_2 / y_test_2.shape[0]
            sum_3_r = 100 - (sum_3 / y_test_3.shape[0] * 100)


            # change res if needed
            if sum_0 < best_0[0]:
                best_0 = [sum_0, kernel, C, eps]

            if sum_1_r < best_1[0]:
                best_1 = [sum_1_r, kernel, C, eps]

            if sum_2 < best_2[0]:
                best_2 = [sum_2, kernel, C, eps]

            if sum_3_r < best_3[0]:
                best_3 = [sum_3_r, kernel, C, eps]

            print(kernel, " ", C, " ", eps,"   - done")
        
print()                
print(best_0)
print(best_1)
print(best_2)
print(best_3)
        '''

In [5]:
#model_regression_0 = SGDRegressor()
#model_regression_0.fit(X_hot, y_train[:,0])
#y_pred_0 = model_regression_0.predict(X_hot_test)
#print("y_pred_0",y_pred_0)
#print("y_pred_0 len", len(y_pred_0))
#model_regression_1 = SGDRegressor()
#model_regression_1.fit(X_hot, y_train[:,1])
#y_pred_1 = model_regression_1.predict(X_hot_test)
#print("y_pred_1",y_pred_1)
#print("y_pred_1 len", len(y_pred_1))
#model_regression_2 = SGDRegressor()
#model_regression_2.fit(X_hot, y_train[:,2])
#y_pred_2 = model_regression_2.predict(X_hot_test)

#model_regression_3 = SGDRegressor()
#model_regression_3.fit(X_hot, y_train[:,3])
#y_pred_3 = model_regression_3.predict(X_hot_test)

In [6]:
y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]
y_test_2 = y_test[:,2]
y_test_3 = y_test[:,3]   

In [7]:
sum_0 = 0
for i in range(y_test_0.shape[0]):
    sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))
sum_0 = sum_0 / y_test_0.shape[0]


print("Number of unique values in y_test[:,0] : ", np.unique(y_test_0).shape[0])
print("Min of y_test[:,0] : ", np.min(y_test_0))
print("Max of y_test[:,0] : ", np.max(y_test_0))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_0).shape[0])
print("Min of y_pred : ", np.min(y_pred_0))
print("Max of y_pred : ", np.max(y_pred_0))
print()
print("Error is on avearage : ", sum_0)
print("Let's say the target value is : 1.0, then the model would guess a value in the following interval :")
print(" -> [", 1.0 - sum_0,",",1.0 + sum_0,"]")


y_pred_0_r = np.around(y_pred_0)

sum_0_r = 0
sum_0_5_r = 0
for i in range(y_pred_0_r.shape[0]):
    sum_0_r = sum_0_r + np.abs(np.abs(y_pred_0_r[i]) - np.abs(y_test_0[i]))
    if(y_pred_0_r[i] == y_test_0[i]):
        sum_0_5_r = sum_0_5_r + 1
sum_0_r = sum_0_r / y_test_0.shape[0]
sum_0_5_r_ = sum_0_5_r / y_test_0.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_0_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_0_r))
print("Max of y_pred_rounded : ", np.max(y_pred_0_r))
print()
print("Error is on avearage : ", sum_0_r)
print("There were ", sum_0_5_r, " correct predictions, out of ", y_test_1.shape[0], ", which is ", 100 - sum_0_5_r_,"% error rate")

Number of unique values in y_test[:,0] :  6400
Min of y_test[:,0] :  -2.8142
Max of y_test[:,0] :  3.7569

Number of unique values in y_pred :  1337
Min of y_pred :  -2.334036396869283
Max of y_pred :  3.311236179902073

Error is on avearage :  0.26732008420568637
Let's say the target value is : 1.0, then the model would guess a value in the following interval :
 -> [ 0.7326799157943136 , 1.2673200842056864 ]
Number of unique values in y_pred_rounded :  6
Min of y_pred_rounded :  -2.0
Max of y_pred_rounded :  3.0

Error is on avearage :  0.3783025520998184
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate


In [8]:
sum_1 = 0
sum_1_5 = 0
for i in range(y_test_1.shape[0]):
    sum_1 = sum_1 + np.abs(np.abs(y_pred_1[i]) - np.abs(y_test_1[i]))
    if(y_pred_1[i] == y_test_1[i]):
        sum_1_5 = sum_1_5 + 1
sum_1 = sum_1 / y_test_1.shape[0]
sum_1_5_r = sum_1_5 / y_test_1.shape[0] * 100

print("Number of unique values in y_test[:,1] : ", np.unique(y_test_1).shape[0])
print("Min of y_test[:,1] : ", np.min(y_test_1))
print("Max of y_test[:,1] : ", np.max(y_test_1))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_1).shape[0])
print("Min of y_pred : ", np.min(y_pred_1))
print("Max of y_pred : ", np.max(y_pred_1))
print()
print("Error is on avearage : ", sum_1)
print("There were ", sum_1_5, " correct predictions, out of ", y_test_1.shape[0], ", which is ", 100 - sum_1_5_r,"% error rate")
print()
print("Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : ")
print()

y_pred_1_r = np.around(y_pred_1)

sum_1_r = 0
sum_1_5_r = 0
for i in range(y_pred_1_r.shape[0]):
    sum_1_r = sum_1_r + np.abs(np.abs(y_pred_1_r[i]) - np.abs(y_test_1[i]))
    if(y_pred_1_r[i] == y_test_1[i]):
        sum_1_5_r = sum_1_5_r + 1
sum_1_r = sum_1_r / y_test_1.shape[0]
sum_1_5_r_ = sum_1_5_r / y_test_1.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_1_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_1_r))
print("Max of y_pred_rounded : ", np.max(y_pred_1_r))
print()
print("Error is on avearage : ", sum_1_r)
print("There were ", sum_1_5_r, " correct predictions, out of ", y_test_1.shape[0], ", which is ", 100 - sum_1_5_r_,"% error rate")

Number of unique values in y_test[:,1] :  7
Min of y_test[:,1] :  0.0
Max of y_test[:,1] :  6.0

Number of unique values in y_pred :  1337
Min of y_pred :  -0.21982706055818502
Max of y_pred :  5.65719387254661

Error is on avearage :  0.6956854528961844
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : 

Number of unique values in y_pred_rounded :  7
Min of y_pred_rounded :  0.0
Max of y_pred_rounded :  6.0

Error is on avearage :  0.6840603857416945
There were  16846  correct predictions, out of  39612 , which is  57.472483085933554 % error rate


In [9]:
sum_2 = 0
sum_2_r = 0
for i in range(y_test_2.shape[0]):
    sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))
    sum_2_r = sum_2_r + np.abs(100 - (np.abs(y_pred_2[i]) * 100 / np.abs(y_test_2[i])))
sum_2 = sum_2 / y_test_2.shape[0]
sum_2_r = sum_2_r / y_test_2.shape[0]


print("Number of unique values in y_test[:,2] : ", np.unique(y_test_2).shape[0])
print("Min of y_test[:,2] : ", np.min(y_test_2))
print("Max of y_test[:,2] : ", np.max(y_test_2))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_2).shape[0])
print("Min of y_pred : ", np.min(y_pred_2))
print("Max of y_pred : ", np.max(y_pred_2))
print()
print("Error is on avearage : ", sum_2)
print("Error rate is on avearage : ", sum_2_r, "% -> for each guess, the model is of by ", sum_2_r, "%")
print("Let's say the target value is : 150, then the model would guess a value in the following interval :")
print(" -> [", 150 - sum_2,",",150 + sum_2,"]")


y_pred_2_r = np.around(y_pred_2)

sum_2_r = 0
sum_2_5_r = 0
for i in range(y_pred_2_r.shape[0]):
    sum_2_r = sum_2_r + np.abs(np.abs(y_pred_2_r[i]) - np.abs(y_test_2[i]))
    if(y_pred_2_r[i] == y_test_2[i]):
        sum_2_5_r = sum_2_5_r + 1
sum_2_r = sum_2_r / y_test_2.shape[0]
sum_2_5_r_ = sum_2_5_r / y_test_2.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_2_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_2_r))
print("Max of y_pred_rounded : ", np.max(y_pred_2_r))
print()
print("Error is on avearage : ", sum_2_r)
print("There were ", sum_2_5_r, " correct predictions, out of ", y_test_2.shape[0], ", which is ", 100 - sum_2_5_r_,"% error rate")


Number of unique values in y_test[:,2] :  222
Min of y_test[:,2] :  104.152
Max of y_test[:,2] :  152.037

Number of unique values in y_pred :  1337
Min of y_pred :  109.11509900242532
Max of y_pred :  133.07554930692822

Error is on avearage :  0.8983270988918186
Error rate is on avearage :  0.6926994034784504 % -> for each guess, the model is of by  0.6926994034784504 %
Let's say the target value is : 150, then the model would guess a value in the following interval :
 -> [ 149.10167290110817 , 150.89832709889183 ]
Number of unique values in y_pred_rounded :  24
Min of y_pred_rounded :  109.0
Max of y_pred_rounded :  133.0

Error is on avearage :  0.9108873691075404
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate


In [10]:


sum_3 = 0
sum_3_5 = 0
for i in range(y_test_3.shape[0]):
    sum_3 = sum_3 + np.abs(np.abs(y_pred_3[i]) - np.abs(y_test_3[i]))
    if(y_pred_3[i] == y_test_3[i]):
        sum_3_5 = sum_3_5 + 1
sum_3 = sum_3 / y_test_1.shape[0]
sum_3_5_r = sum_3_5 / y_test_1.shape[0] * 100

print("Number of unique values in y_test[:,3] : ", np.unique(y_test_3).shape[0])
print("Min of y_test[:,3] : ", np.min(y_test_3))
print("Max of y_test[:,3] : ", np.max(y_test_3))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_3).shape[0])
print("Min of y_pred : ", np.min(y_pred_3))
print("Max of y_pred : ", np.max(y_pred_3))
print()
print("Error is on avearage : ", sum_3)
print("There were ", sum_3_5, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_3_5_r,"% error rate")
print()
print("Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : ")
print()

y_pred_3_r = np.around(y_pred_3)

sum_3_r = 0
sum_3_5_r = 0
for i in range(y_test_3.shape[0]):
    sum_3_r = sum_3_r + np.abs(np.abs(y_pred_3_r[i]) - np.abs(y_test_3[i]))
    if(y_pred_3_r[i] == y_test_3[i]):
        sum_3_5_r = sum_3_5_r + 1
sum_3_r = sum_3_r / y_test_3.shape[0]
sum_3_5_r_ = sum_3_5_r / y_test_3.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_3_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_3_r))
print("Max of y_pred_rounded : ", np.max(y_pred_3_r))
print()
print("Error is on avearage : ", sum_3_r)
print("There were ", sum_3_5_r, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_3_5_r_,"% error rate")

Number of unique values in y_test[:,3] :  9
Min of y_test[:,3] :  0.0
Max of y_test[:,3] :  8.0

Number of unique values in y_pred :  1337
Min of y_pred :  -0.20036918729748265
Max of y_pred :  5.800333160580948

Error is on avearage :  0.17144155932471927
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : 

Number of unique values in y_pred_rounded :  7
Min of y_pred_rounded :  -0.0
Max of y_pred_rounded :  6.0

Error is on avearage :  0.08467131172372008
There were  36724  correct predictions, out of  39612 , which is  7.290719983843275 % error rate


In [12]:
import json #ne jamais faire ça avec les import 

def load_dict_errors(path="all_errors.json"):
    with open(path, "r") as f:
        dict_errors = json.load(f)
        
    return dict_errors

dict_errors = load_dict_errors()
print("dict_errors, after loading : ",dict_errors)



def save_dict_errors(dict_errors, path="all_errors.json"):
    with open(path, "w") as f:
        json.dump(dict_errors, f)

dict_errors['SVR'] = [0, 100 - sum_1_5_r_, 0, 100 - sum_3_5_r_]

#dict_errors = { "GradBoostRegressor" : [100 - sum_0_5_r_, 100 - sum_1_5_r_, 100 - sum_2_5_r_, 100 - sum_3_5_r_]}

print("dict_errors, after changes : ",dict_errors)
save_dict_errors(dict_errors)

dict_errors, after loading :  {'GradBoostRegressor': [0, 64.05887104917701, 0, 6.879228516611121], 'LinearRegression': [100.0, 57.472483085933554, 100.0, 7.290719983843275]}
dict_errors, after changes :  {'GradBoostRegressor': [0, 64.05887104917701, 0, 6.879228516611121], 'LinearRegression': [100.0, 57.472483085933554, 100.0, 7.290719983843275], 'SVR': [0, 57.472483085933554, 0, 7.290719983843275]}


In [13]:
import json #ne jamais faire ça avec les import 

def load_dict_errors(path="all_errors.json"):
    with open(path, "r") as f:
        dict_errors = json.load(f)
        
    return dict_errors

dict_errors_0_2 = load_dict_errors(path="all_errors_0_2.json")
dict_errors_1_3 = load_dict_errors(path="all_errors_1_3.json")
print("dict_errors_0_2", dict_errors_0_2)
print("dict_errors_1_3", dict_errors_1_3)


def save_dict_errors(dict_errors, path="all_errors.json"):
    with open(path, "w") as f:
        json.dump(dict_errors, f)

# kernel restarted, too long to retrain all the data for a model we won't use, copy pasted the above results.
dict_errors_0_2['SVR'] = [0.26732008420568637, 0.8983270988918186]
dict_errors_1_3['SVR'] = [57.472483085933554, 7.290719983843275]

print("dict_errors, after changes : ",dict_errors_0_2)
print("dict_errors, after changes : ",dict_errors_1_3)

save_dict_errors(dict_errors_0_2, path="all_errors_0_2.json")
save_dict_errors(dict_errors_1_3, path="all_errors_1_3.json")

dict_errors_0_2 {'lienar_regression': [0.34321702049967723, 0.017421293770233204], 'neural_network': [0.26404827656252755, 2.9858224688390417e-06], 'SGDRegressor': [0.35036686833289443, 0.010050606676439778], 'GradBoostRegressor': [0.3059346708449163, 3.882566534691418]}
dict_errors_1_3 {'GradBoostRegressor': [64.05887104917701, 6.879228516611121], 'SVR': [57.472483085933554, 7.290719983843275], 'lienar_regression': [56.280924972230636, 5.710390790669493], 'neural_network': [45.78915480157528, 35.03483792790064], 'neer_neighboor': [54.907603756437446, 7.081187518933646], 'SGDRegressor': [56.52327577501767, 7.01050186812077]}
dict_errors, after changes :  {'lienar_regression': [0.34321702049967723, 0.017421293770233204], 'neural_network': [0.26404827656252755, 2.9858224688390417e-06], 'SGDRegressor': [0.35036686833289443, 0.010050606676439778], 'GradBoostRegressor': [0.3059346708449163, 3.882566534691418], 'SVR': [0.26732008420568637, 0.8983270988918186]}
dict_errors, after changes :  {